In [547]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

# Create Table and Feature Selection
table_full = pd.read_csv('table_full.csv')

table_full = table_full.loc[:, ]

progRank = table_full['Progression RANK'].values


# Create Dummies Varibles
dummies = pd.get_dummies(table_full[['region', 'countries']])
table_full = pd.concat([table_full, dummies], axis=1)

del table_full['countries']
del table_full['region']

# Creating Y
est = KBinsDiscretizer(n_bins=2, encode='ordinal', strategy='uniform')
progreesinRank = progRank.reshape(-1,1)
est.fit(progreesinRank)
y = est.transform(progreesinRank).astype(np.int64)

###################################################################################
#With Robust
robustscaler = RobustScaler()
table_full_robusted = robustscaler.fit_transform(table_full)
table_full_robusted = pd.DataFrame(table_full_robusted, columns=table_full.columns)
list_X = list(table_full.columns)
X = table_full_robusted[list(list_X)].values

#del table_full_robusted['Progression RANK']

# Without Robust
# del table_full_robusted['Progression RANK']

# list_X = list(table_full.columns)
# X = table_full[list(list_X)].values

###################################################################################
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# KNN MOdel Creation
knn = KNeighborsClassifier()

knn.fit(X_train, y_train.ravel())
print('Accuracy of K-NN classifier on training set: {:.2f}'.format(knn.score(X_train, y_train)))
print('Accuracy of K-NN classifier on test set: {:.2f}'.format(knn.score(X_test, y_test)))

# Evaluation
y_pred_class = knn.predict(X_test)

confusion = metrics.confusion_matrix(y_test, y_pred_class)
print(confusion)

print(metrics.classification_report(y_test, y_pred_class, digits=3))

Accuracy of K-NN classifier on training set: 0.68
Accuracy of K-NN classifier on test set: 0.61
[[17  0]
 [12  2]]
              precision    recall  f1-score   support

           0      0.586     1.000     0.739        17
           1      1.000     0.143     0.250        14

    accuracy                          0.613        31
   macro avg      0.793     0.571     0.495        31
weighted avg      0.773     0.613     0.518        31

